In [2]:
# Disable optional backends to prevent import conflicts (PyTorch-only training)
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"
os.environ["TRANSFORMERS_NO_TORCHVISION"] = "1"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
# Reduce CUDA fragmentation for large models
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
print("✅ Env set: disabled TF/FLAX/TORCHVISION for transformers.")
print("✅ CUDA alloc conf: expandable_segments enabled.")

✅ Env set: disabled TF/FLAX/TORCHVISION for transformers.
✅ CUDA alloc conf: expandable_segments enabled.


# 🚀 GIS代码生成模型训练 - Google Colab (CodeLlama)

本Notebook在Google Colab上训练GIS代码生成模型（**文件级 + CodeLlama**）

**文件级训练** - 模型学习生成完整的工作流而不是单个步骤
- 输入：用户的高层指令（英语/荷兰语，如："Create MS and HS cable objects"）
- 输出：完整的工作流JSON代码（包含所有操作步骤）
- 优势：一次推理生成整个测试脚本
- **模型**：CodeLlama-7B-Instruct（专为代码生成优化）

**使用前准备：**
1. 运行环境：`Runtime > Change runtime type > T4 GPU`（免费）或 `A100 GPU`（Colab Pro）
2. 数据准备：确保已生成训练数据文件
3. 预计时间：4-6小时（T4）/ 1-2小时（A100）

---

## 📋 步骤1：环境设置

In [3]:
# 检查GPU
!nvidia-smi

Mon Jan 19 09:08:15 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# 安装依赖（约3-5分钟）
print("📦 Installing dependencies...")

# 先锁定关键基础包（避免自动升级）
!pip install -q torch==2.9.0 --no-deps
!pip install -q fsspec==2024.3.1
!pip install -q numpy==2.0.2 --no-deps

# 安装主要训练库（指定兼容版本）
!pip install -q transformers==4.46.0
!pip install -q peft==0.13.0
!pip install -q datasets==2.19.0
!pip install -q "accelerate>=1.0.0"
!pip install -q sentencepiece==0.2.0
!pip install -q tqdm
!pip install -q huggingface-hub==0.26.0

print("✅ Core dependencies installed! If running in Colab, restart runtime after this cell.")

📦 Installing dependencies...
✅ Core dependencies installed! If running in Colab, restart runtime after this cell.


In [8]:
!pip install -q torchvision

## 💾 步骤2：挂载Google Drive（保存模型）

In [5]:
from google.colab import drive
drive.mount('/content/drive')

# 创建输出目录
!mkdir -p /content/drive/MyDrive/gis-models
print("✅ Google Drive mounted!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted!


## 📂 步骤3：上传数据

**两种方式任选其一：**

### 方式A：从GitHub克隆仓库（推荐）

In [6]:
# 克隆仓库到当前目录
import os
import subprocess

print("📍 Current directory:", os.getcwd())

# 检查是否已克隆
if os.path.exists('/content/gis-code-ai'):
    print("✅ Repository already exists at /content/gis-code-ai")
    os.chdir('/content/gis-code-ai')
elif os.path.exists('gis-code-ai'):
    print("✅ Repository already exists locally")
    os.chdir('gis-code-ai')
else:
    print("🔄 Cloning repository...")
    result = subprocess.run(
        ['git', 'clone', 'https://github.com/rockyistt/gis-code-ai.git'],
        cwd='/content',
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        os.chdir('/content/gis-code-ai')
        print("✅ Repository cloned successfully")
    else:
        print(f"❌ Clone failed: {result.stderr}")
        raise RuntimeError("Failed to clone repository")

print(f"📍 Working directory: {os.getcwd()}")

# 检查数据文件
print("\n📋 Checking data files...")
required_files = [
    'data/processed/file_level_instructions_weighted_variants_marked.jsonl',
    'data/processed/parsed_workflows.jsonl'
]

all_exist = True
for f in required_files:
    if os.path.exists(f):
        size_mb = os.path.getsize(f) / (1024*1024)
        print(f"✅ {f} ({size_mb:.1f} MB)")
    else:
        print(f"❌ {f}")
        all_exist = False

if all_exist:
    print("\n🎉 All data files ready! Ready to proceed with data preparation")
else:
    print("\n⚠️ Some data files are missing")

📍 Current directory: /content
✅ Repository already exists at /content/gis-code-ai
📍 Working directory: /content/gis-code-ai

📋 Checking data files...
✅ data/processed/file_level_instructions_weighted_variants_marked.jsonl (2.1 MB)
✅ data/processed/parsed_workflows.jsonl (13.8 MB)

🎉 All data files ready! Ready to proceed with data preparation


## 🔄 步骤4：准备训练数据

In [ ]:
import json
from typing import Tuple, List, Dict, Optional

def prepare_file_level_training_data(
    instructions_file: str,
    workflows_file: str,
    output_dir: str = 'data/training',
    max_samples: Optional[int] = None,
    split_ratio: float = 0.9
) -> Tuple[List[Dict], List[Dict]]:
    """
    准备文件级训练数据
    - 输入：file_level_instructions_weighted_variants_marked.jsonl
    - 输出：parsed_workflows.jsonl 中对应的原始JSON
    """
    import os
    import json
    from pathlib import Path
    
    os.makedirs(output_dir, exist_ok=True)
    
    # 加载指令
    instructions = {}
    print(f"📖 Loading instructions from {instructions_file}...")
    with open(instructions_file, 'r', encoding='utf-8') as f:
        for line in f:
            instr = json.loads(line)
            file_id = instr.get('file_id')
            if file_id:
                instructions[file_id] = instr
    print(f"✅ Loaded {len(instructions)} instructions")
    
    # 加载工作流
    workflows = {}
    print(f"📖 Loading workflows from {workflows_file}...")
    with open(workflows_file, 'r', encoding='utf-8') as f:
        for line in f:
            workflow = json.loads(line)
            file_id = workflow.get('file_id')
            if file_id:
                workflows[file_id] = workflow
    print(f"✅ Loaded {len(workflows)} workflows")
    
    # 配对并创建训练数据
    training_pairs = []
    print("🔄 创建训练对...")
    
    for file_id, instr in instructions.items():
        if file_id not in workflows:
            continue
        
        workflow = workflows[file_id]
        
        # 清理指令
        instruction_text = instr.get('instruction', '').replace('**', '').replace('*', '')
        instruction_text = ' '.join(instruction_text.split()).strip()
        
        # 构建上下文
        test_app = workflow.get('test_app', '')
        database = workflow.get('database', '')
        total_steps = workflow.get('total_steps', 0)
        
        context_parts = []
        if test_app:
            context_parts.append(f"Application: {test_app}")
        if database:
            context_parts.append(f"Database: {database}")
        if total_steps > 0:
            context_parts.append(f"Steps: {total_steps}")
        input_context = " | ".join(context_parts)
        
        # ✅ 直接使用 parsed_workflows.jsonl 中的原始 JSON
        if not workflow.get('steps'):
            continue
        
        # 直接使用 workflow 对象作为输出
        output_code = json.dumps(workflow, indent=2, ensure_ascii=False)
        
        # 质量过滤
        if len(instruction_text.split()) < 5 or output_code == '{}':
            continue
        
        training_pairs.append({
            'instruction': instruction_text,
            'input': input_context,
            'output': output_code
        })
    
    if max_samples:
        training_pairs = training_pairs[:max_samples]
    
    print(f"✅ Created {len(training_pairs)} training pairs")
    
    # 数据集分割
    split_point = int(len(training_pairs) * split_ratio)
    train_data = training_pairs[:split_point]
    val_data = training_pairs[split_point:]
    
    # 保存为JSON文件
    train_file = os.path.join(output_dir, 'training_data_train.json')
    val_file = os.path.join(output_dir, 'training_data_val.json')
    
    with open(train_file, 'w', encoding='utf-8') as f:
        json.dump(train_data, f, ensure_ascii=False, indent=2)
    print(f"💾 Train data saved to {train_file}")
    
    with open(val_file, 'w', encoding='utf-8') as f:
        json.dump(val_data, f, ensure_ascii=False, indent=2)
    print(f"💾 Val data saved to {val_file}")
    
    return train_data, val_data

print("✅ prepare_file_level_training_data 函数已定义")

✅ Data preparation function defined


In [10]:
# 运行数据准备（直接调用函数）
import os

# 确保在正确的目录
if os.path.exists('/content/gis-code-ai'):
    os.chdir('/content/gis-code-ai')
elif os.path.exists('gis-code-ai'):
    os.chdir('gis-code-ai')

print("="*70)
print("🔄 开始数据准备...")
print(f"📍 Working directory: {os.getcwd()}")
print("="*70)

train_data, val_data = prepare_file_level_training_data(
    instructions_file='data/processed/file_level_instructions_weighted_variants_marked.jsonl',
    workflows_file='data/processed/parsed_workflows.jsonl',
    output_dir='data/training',
    max_samples=None,  # 使用全部数据，改为整数可限制样本数（如500）
    split_ratio=0.9
)

if train_data is not None:
    print("\n" + "="*70)
    print("🎉 数据准备完成！")
    print(f"📊 数据粒度: 文件级（完整工作流）")
    print(f"📊 训练样本: {len(train_data):,}")
    print(f"📊 验证样本: {len(val_data):,}")
    print("="*70)

    # 显示示例
    if train_data:
        print("\n📝 训练样本示例:")
        sample = train_data[0]
        print(f"Instruction: {sample['instruction'][:100]}...")
        print(f"Input: {sample['input']}")
        print(f"Output preview: {sample['output'][:200]}...")
else:
    print("\n❌ 数据准备失败！请检查上面的错误信息。")

🔄 开始数据准备...
📍 Working directory: /content/gis-code-ai
📖 Loading file-level instructions from data/processed/file_level_instructions_weighted_variants_marked.jsonl
✅ Loaded 1012 file-level instructions
📖 Loading workflows from data/processed/parsed_workflows.jsonl
✅ Loaded 1012 workflows
🔄 Converting to training format (file-level)...


100%|██████████| 1012/1012 [00:00<00:00, 1901.73it/s]

✅ Created 1012 training samples
📊 Split: 910 train, 102 validation
💾 Train data saved: data/training/training_data_train.json
💾 Validation data saved: data/training/training_data_val.json

🎉 数据准备完成！
📊 数据粒度: 文件级（完整工作流）
📊 训练样本: 910
📊 验证样本: 102

📝 训练样本示例:
Instruction: Workflow: create E MS Kabel, E HS Kabel, E LS Kabel in elektra in NRG Beheerkaart Elektra MS...
Input: Application: NRG Beheerkaart Elektra MS | Steps: 7
Output preview: {
  "workflow": {
    "metadata": {
      "test_app": "NRG Beheerkaart Elektra MS",
      "database": "",
      "total_steps": 7
    },
    "steps": [
      {
        "step_index": 0,
        "databas...


## 🚀 步骤5：开始训练

In [11]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import load_dataset

print("🔧 Setting up training with CodeLlama...")

# ============================================================
# 配置参数 - CodeLlama (英语/荷兰语代码生成优化)
# ============================================================

MODEL_NAME = "codellama/CodeLlama-7b-Instruct-hf"
OUTPUT_DIR = "/content/drive/MyDrive/gis-models/codellama-gis-lora"  # 保存到Google Drive
TRAIN_FILE = "data/training/training_data_train.json"
VAL_FILE = "data/training/training_data_val.json"

# 训练参数（T4 GPU内存优化版 - 14GB显存限制）
NUM_EPOCHS = 3
BATCH_SIZE = 1  # Per-device batch size
GRADIENT_ACCUMULATION = 2  # Reduced from 4 → effective batch = 2
LEARNING_RATE = 2e-4  # CodeLlama推荐学习率
MAX_LENGTH = 512  # Reduced from 768 → significant memory savings

# LoRA参数（代码生成任务优化 - T4优化版）
LORA_R = 32  # Reduced from 64 → less trainable parameters
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

print(f"📦 Model: {MODEL_NAME}")
print(f"💾 Output: {OUTPUT_DIR}")
print(f"📊 Data: FILE-LEVEL (complete workflows)")
print(f"📈 Epochs: {NUM_EPOCHS}, Batch: {BATCH_SIZE}, Accumulation: {GRADIENT_ACCUMULATION}, LR: {LEARNING_RATE}")
print(f"📄 Max Length: {MAX_LENGTH}")
print(f"🎯 LoRA Rank: {LORA_R} (T4-optimized)")
print(f"🎯 Optimized for: T4 GPU (14GB VRAM) - memory-efficient config")

🔧 Setting up training with CodeLlama...
📦 Model: codellama/CodeLlama-7b-Instruct-hf
💾 Output: /content/drive/MyDrive/gis-models/codellama-gis-lora
📊 Data: FILE-LEVEL (complete workflows)
📈 Epochs: 3, Batch: 1, Accumulation: 2, LR: 0.0002
📄 Max Length: 512
🎯 LoRA Rank: 32 (T4-optimized)
🎯 Optimized for: T4 GPU (14GB VRAM) - memory-efficient config


In [12]:
# 加载tokenizer (CodeLlama)
print("📖 Loading CodeLlama tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    padding_side="right"
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"✅ Tokenizer loaded: vocab_size={len(tokenizer)}")

📖 Loading CodeLlama tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Tokenizer loaded: vocab_size=32016


In [13]:
# ⚠️ 必须在model loading前运行！升级关键库
print("🔧 Upgrading transformers, accelerate, and installing bitsandbytes...")

# 升级transformers到兼容版本
!pip install -q --upgrade transformers==4.46.0

# 升级accelerate到最新稳定版修复optimizer.train()错误
!pip install -q --upgrade accelerate>=1.0.0

# 安装bitsandbytes用于8-bit优化器（节省显存）
!pip install -q bitsandbytes

print("✅ Libraries upgraded!")
print(f"  transformers: 4.46.0")
print(f"  accelerate: >=1.0.0 (latest stable, fixes optimizer.train() bug)")
print(f"  bitsandbytes: installed (8-bit optimizer for memory efficiency)")

🔧 Upgrading transformers, accelerate, and installing bitsandbytes...
✅ Libraries upgraded!
  transformers: 4.46.0
  accelerate: >=1.0.0 (latest stable, fixes optimizer.train() bug)
  bitsandbytes: installed (8-bit optimizer for memory efficiency)
⚠️ IMPORTANT: Runtime restart recommended!
   Please go to: Runtime > Restart runtime
   Then re-run all cells in order


In [14]:
# 确保导入了必要的库
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
import torch

# 使用float16加载模型（不量化，稳定适配T4显存）
print("🤖 Loading CodeLlama-7B with float16 and device offloading...")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True,
)

# 训练中必须禁用缓存以配合梯度检查点
model.config.use_cache = False

# 启用梯度检查点（节省显存）
model.gradient_checkpointing_enable()

print("✅ CodeLlama base model loaded (float16 with device offloading)")

# 应用LoRA
print("🔧 Applying LoRA...")
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("✅ LoRA applied!")

🤖 Loading CodeLlama-7B with float16 and device offloading...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ CodeLlama base model loaded (float16 with device offloading)
🔧 Applying LoRA...
trainable params: 79,953,920 || all params: 6,818,500,608 || trainable%: 1.1726
✅ LoRA applied!


In [ ]:
from datasets import load_dataset

# 准备数据集
print("📊 Preparing datasets...")

train_data = load_dataset('json', data_files=TRAIN_FILE, split='train')
eval_data = load_dataset('json', data_files=VAL_FILE, split='train')

print(f"  Train: {len(train_data)} samples")
print(f"  Val: {len(eval_data)} samples")

# 格式化prompt (CodeLlama优化格式)
def format_prompt(example):
    instruction = example['instruction']
    input_text = example.get('input', '')
    output = example['output']

    # CodeLlama更适合直接的代码生成格式
    if input_text:
        prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}
Context: {input_text}

JSON Code:
{output}"""
    else:
        prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}

JSON Code:
{output}"""

    return {"text": prompt}

train_data = train_data.map(format_prompt, remove_columns=train_data.column_names)
eval_data = eval_data.map(format_prompt, remove_columns=eval_data.column_names)

# Tokenize
def tokenize_function(examples):
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,
        return_tensors=None
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("🔄 Tokenizing...")
train_dataset = train_data.map(
    tokenize_function,
    batched=True,
    remove_columns=train_data.column_names,
    desc="Tokenizing train"
)

eval_dataset = eval_data.map(
    tokenize_function,
    batched=True,
    remove_columns=eval_data.column_names,
    desc="Tokenizing val"
)

print("✅ Datasets ready!")

📊 Preparing datasets...


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

  Train: 910 samples
  Val: 102 samples


Map:   0%|          | 0/910 [00:00<?, ? examples/s]

Map:   0%|          | 0/102 [00:00<?, ? examples/s]

🔄 Tokenizing...


Tokenizing train:   0%|          | 0/910 [00:00<?, ? examples/s]

Tokenizing val:   0%|          | 0/102 [00:00<?, ? examples/s]

✅ Datasets ready!


In [16]:
# 配置训练
print("⚙️ Configuring training (T4-optimized)...")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    warmup_steps=50,  # Reduced from 100
    logging_steps=10,
    save_steps=300,  # Reduced from 500
    eval_steps=300,  # Reduced from 500
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    fp16=True,  # Enable fp16 compute
    bf16=False,
    optim="adamw_8bit",  # 8-bit AdamW for memory efficiency (requires bitsandbytes)
    lr_scheduler_type="cosine",
    save_total_limit=2,  # Reduced from 3 to save disk space
    report_to="none",
    logging_dir=f"{OUTPUT_DIR}/logs",
    ddp_find_unused_parameters=False,
    remove_unused_columns=False,
    push_to_hub=False,
    gradient_checkpointing=True,  # Memory savings via checkpointing
    max_grad_norm=1.0,  # Gradient clipping
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,  # Use processing_class instead of tokenizer (new API)
    data_collator=data_collator,
)

print("✅ Trainer ready!")
print("\n" + "="*70)
print("🚀 Starting training (T4-optimized, may take 1-2 hours)...")
print("="*70)

# 训练前清理显存，减少碎片
import gc
gc.collect()
torch.cuda.empty_cache()

# 开始训练
trainer.train()

print("\n" + "="*70)
print("🎉 Training completed!")
print("="*70)

⚙️ Configuring training (T4-optimized)...
✅ Trainer ready!

🚀 Starting training (T4-optimized, may take 6-8 hours)...


Step,Training Loss,Validation Loss
300,0.050500,0.047387
600,0.043000,0.044602
900,0.047200,0.044234
1200,0.037500,0.043698



🎉 Training completed!


In [17]:
# 保存模型
print("💾 Saving model...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"✅ Model saved to {OUTPUT_DIR}")

# 保存训练信息
import json
training_info = {
    "model_name": MODEL_NAME,
    "num_epochs": NUM_EPOCHS,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "lora_r": LORA_R,
    "lora_alpha": LORA_ALPHA,
    "train_samples": len(train_dataset),
    "val_samples": len(eval_dataset),
}

with open(f"{OUTPUT_DIR}/training_info.json", 'w') as f:
    json.dump(training_info, f, indent=2)

print("\n📊 Training Summary:")
for key, value in training_info.items():
    print(f"  {key}: {value}")

💾 Saving model...
✅ Model saved to /content/drive/MyDrive/gis-models/codellama-gis-lora

📊 Training Summary:
  model_name: codellama/CodeLlama-7b-Instruct-hf
  num_epochs: 3
  batch_size: 1
  learning_rate: 0.0002
  lora_r: 32
  lora_alpha: 16
  train_samples: 910
  val_samples: 102


In [ ]:
# 🧪 验证集快速抽样测试（查看生成效果）
import torch, json, textwrap
from random import sample as random_sample

print("\n" + "="*70)
print("🧪 Quick sanity check on validation set (5 samples)")
print("="*70)

model.eval()

# 转换验证集为列表方便索引
eval_data_list = [eval_data[i] for i in range(len(eval_data))]
SAMPLE_N = min(5, len(eval_data_list))
indices = random_sample(range(len(eval_data_list)), SAMPLE_N)

for idx in indices:
    item = eval_data_list[int(idx)]
    instruction = item.get("instruction", "")
    context = item.get("input", "")
    prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}
Context: {context}

JSON Code:
"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 仅截取JSON部分
    if "JSON Code:" in generated:
        generated = generated.split("JSON Code:")[-1].strip()
    print("\n" + "-"*70)
    print(f"Sample #{idx}")
    print("Instruction:", textwrap.shorten(instruction, width=140))
    print("Context:", textwrap.shorten(context, width=140))
    print("Generated (truncated):")
    print(textwrap.shorten(generated.replace("\n", " "), width=400))

In [ ]:
# 📊 结构化成功率指标（验证集）
import numpy as np

print("\n" + "="*70)
print("📊 Validation set structural success metrics")
print("="*70)

REQUIRED_TOP_KEYS = ["workflow"]
REQUIRED_WORKFLOW_KEYS = ["steps"]
REQUIRED_STEP_KEYS = ["type", "params"]

def extract_json(text):
    if "{" not in text:
        return text
    start = text.find("{")
    end = text.rfind("}")
    return text[start:end+1] if end >= start else text

def validate_one(text):
    result = {"json_ok": False, "ok": False, "missing_top": [], "missing_workflow": [], "missing_steps": [], "steps": 0}
    try:
        obj = json.loads(text)
        result["json_ok"] = True
    except Exception:
        return result
    missing_top = [k for k in REQUIRED_TOP_KEYS if k not in obj]
    wf = obj.get("workflow", {})
    missing_wf = [k for k in REQUIRED_WORKFLOW_KEYS if k not in wf]
    steps = wf.get("steps", []) if isinstance(wf, dict) else []
    step_missing = []
    for s in steps:
        if isinstance(s, dict):
            miss = [k for k in REQUIRED_STEP_KEYS if k not in s]
        else:
            miss = REQUIRED_STEP_KEYS
        step_missing.append(miss)
    result.update({
        "missing_top": missing_top,
        "missing_workflow": missing_wf,
        "missing_steps": step_missing,
        "steps": len(steps),
    })
    ok = (len(missing_top)==0 and len(missing_wf)==0 and all(len(m)==0 for m in step_missing) and len(steps)>0)
    result["ok"] = ok
    return result

MAX_EVAL = min(100, len(eval_data))
success = 0
json_ok = 0
step_counts = []
fail_reasons = []

for i in range(MAX_EVAL):
    item = eval_data[int(i)]
    instruction = item.get("instruction", "")
    context = item.get("input", "")
    prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}
Context: {context}

JSON Code:
"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "JSON Code:" in text:
        text = text.split("JSON Code:")[-1].strip()
    text = extract_json(text)
    res = validate_one(text)
    json_ok += int(res["json_ok"])
    success += int(res["ok"])
    step_counts.append(res.get("steps", 0))
    if not res["ok"]:
        reason = []
        if not res["json_ok"]: reason.append("json_parse")
        if res["missing_top"]: reason.append(f"miss_top:{','.join(res['missing_top'])}")
        if res["missing_workflow"]: reason.append(f"miss_workflow:{','.join(res['missing_workflow'])}")
        if any(res["missing_steps"]): reason.append("miss_step_fields")
        if res.get("steps",0)==0: reason.append("no_steps")
        fail_reasons.append(",".join(reason) or "unknown")

N = MAX_EVAL
json_rate = json_ok / N if N else 0
success_rate = success / N if N else 0
avg_steps = float(np.mean(step_counts)) if step_counts else 0
print(f"Samples evaluated: {N}")
print(f"JSON parse success: {json_rate:.1%}")
print(f"Full structural success: {success_rate:.1%}")
print(f"Average steps (all): {avg_steps:.2f}")

from collections import Counter
if fail_reasons:
    top_fail = Counter(fail_reasons).most_common(5)
    print("Top failure reasons:")
    for r,c in top_fail:
        print(f"  {r}: {c}")
else:
    print("No failures recorded")

## 🧪 步骤6：在测试集上评估模型

In [ ]:
# 📊 在测试集上评估模型性能
import json
import numpy as np
from pathlib import Path
from collections import Counter

print("\n" + "="*70)
print("📊 模型性能评估 - 测试集")
print("="*70)

# ===================================================================
# 加载测试数据
# ===================================================================
print("\n📂 加载测试数据...")

# 使用验证集列表作为测试集
test_data_list = [eval_data_list[i] for i in range(len(eval_data_list))]
NUM_EVAL_SAMPLES = min(50, len(test_data_list))  # 评估前50个样本

print(f"✅ 加载了 {len(test_data_list)} 个验证样本")
print(f"📊 将评估前 {NUM_EVAL_SAMPLES} 个样本\n")

# ===================================================================
# 定义评估指标函数
# ===================================================================
def is_valid_json(text):
    """检查文本是否是有效的JSON"""
    try:
        json.loads(text)
        return True
    except (json.JSONDecodeError, TypeError):
        return False

def get_step_count(text):
    """从JSON中提取步骤数"""
    try:
        obj = json.loads(text)
        steps = obj.get("workflow", {}).get("steps", [])
        return len(steps)
    except:
        return 0

def extract_json_from_text(text):
    """从生成的文本中提取JSON部分"""
    # 尝试找到JSON的开始
    if "{" in text:
        start = text.index("{")
        # 尝试找到JSON的结束
        if "}" in text[start:]:
            end = text.rindex("}") + 1
            return text[start:end]
    return text

# ===================================================================
# 执行批量评估
# ===================================================================
print("🧪 开始评估...")
print("="*70)

evaluation_results = {
    "json_valid": [],
    "step_counts": [],
    "instruction_lengths": [],
    "generation_successful": [],
}

json_valid_count = 0
successful_count = 0

for i in range(NUM_EVAL_SAMPLES):
    sample = test_data_list[i]
    instruction = sample.get("instruction", "")
    context = sample.get("input", "")
    
    # 显示进度
    if (i + 1) % 10 == 0:
        print(f"进度: {i + 1}/{NUM_EVAL_SAMPLES}")
    
    try:
        # 构建prompt
        prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}
Context: {context}

JSON Code:
"""
        
        # Tokenize
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
        # 生成
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=256,  # 限制长度加快速度
                temperature=0.7,
                do_sample=True,
                top_p=0.9,
                pad_token_id=tokenizer.pad_token_id,
            )
        
        # 解码
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 提取JSON部分
        if "JSON Code:" in generated_text:
            generated_code = generated_text.split("JSON Code:")[-1].strip()
        else:
            generated_code = generated_text
        
        generated_code = extract_json_from_text(generated_code)
        
        # 评估结果
        is_valid = is_valid_json(generated_code)
        step_count = get_step_count(generated_code) if is_valid else 0
        
        evaluation_results["generation_successful"].append(True)
        evaluation_results["json_valid"].append(is_valid)
        evaluation_results["step_counts"].append(step_count)
        evaluation_results["instruction_lengths"].append(len(instruction.split()))
        
        if is_valid:
            json_valid_count += 1
        
        successful_count += 1
    
    except Exception as e:
        # 生成失败
        evaluation_results["generation_successful"].append(False)
        evaluation_results["json_valid"].append(False)
        evaluation_results["step_counts"].append(0)
        evaluation_results["instruction_lengths"].append(len(instruction.split()))
        print(f"  ⚠️ 样本 {i+1} 生成失败: {str(e)[:50]}")

# ===================================================================
# 计算并显示评估结果
# ===================================================================
print("\n" + "="*70)
print("📊 评估结果摘要")
print("="*70)

total_samples = len(evaluation_results["generation_successful"])

# 1. 生成成功率
print(f"\n✅ 生成成功率:")
print(f"   成功: {successful_count}/{total_samples} ({successful_count/total_samples:.1%})")

# 2. JSON有效性
if successful_count > 0:
    print(f"\n✅ JSON有效性 (成功生成的代码中):")
    print(f"   有效: {json_valid_count}/{successful_count} ({json_valid_count/successful_count:.1%})")
else:
    print(f"\n❌ 所有生成均失败")

# 3. 步骤数统计
valid_step_counts = [s for s, v in zip(evaluation_results["step_counts"], evaluation_results["json_valid"]) if v]
if valid_step_counts:
    print(f"\n📍 有效代码的步骤数统计:")
    print(f"   平均: {np.mean(valid_step_counts):.1f}")
    print(f"   中位: {np.median(valid_step_counts):.1f}")
    print(f"   范围: {int(np.min(valid_step_counts))}-{int(np.max(valid_step_counts))}")

# 4. 指令长度分析
print(f"\n📝 指令长度 (词数):")
print(f"   平均: {np.mean(evaluation_results['instruction_lengths']):.1f}")
print(f"   最长: {int(np.max(evaluation_results['instruction_lengths']))}")

# 5. 总体质量评估
if successful_count > 0:
    quality_rate = json_valid_count / successful_count
else:
    quality_rate = 0

quality_stars = (
    "⭐⭐⭐⭐⭐" if quality_rate > 0.9 else
    "⭐⭐⭐⭐" if quality_rate > 0.7 else
    "⭐⭐⭐" if quality_rate > 0.5 else
    "⭐⭐" if quality_rate > 0.3 else
    "⭐"
)

print(f"\n🎯 总体质量评估:")
print(f"   {quality_stars} ({quality_rate:.1%} 有效率)")

# ===================================================================
# 改进建议
# ===================================================================
print("\n" + "="*70)
print("💡 改进建议")
print("="*70)

if quality_rate < 0.5:
    print("""
⚠️  模型质量较低，建议:
   1. 增加训练轮数 (num_epochs: 3 → 5)
   2. 使用更多训练数据
   3. 降低学习率 (lr: 2e-4 → 1e-4)
   4. 增加预热步骤 (warmup_steps: 50 → 200)
   5. 尝试不同的LoRA秩 (r: 32 → 64)
    """)
elif quality_rate < 0.7:
    print("""
✓ 模型质量一般，可以继续改进:
   1. 增加训练轮数
   2. 微调学习率
   3. 收集更多多样化的训练样本
    """)
else:
    print("""
✅ 模型质量良好！
   • 可以开始在真实场景中使用
   • 继续收集失败样例以进一步改进
   • 考虑在其他GIS工作流上进行微调
    """)

print("\n" + "="*70)
print("🎉 评估完成！")
print("="*70)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


🧪 Testing CodeLlama model inference...

🔮 Generating...

📝 Test Result:
Instruction: Create a new MS cable object at coordinates (186355533, 439556907)
Context: Application: PowerGrid | Database: ND | Steps: 5

Generated Output:
{
  "workflow": {
    "metadata": {
      "test_app": "PowerGrid",
      "database": "ND",
      "total_steps": 5
    },
    "steps": [
      {
        "step_index": 0,
        "database": ":catalog",
        "object": "Object Control",
        "object_id": "Passed",
        "module": "Tabs",
        "method": "Select Tab",
        "command": "Execute Tabs Testcommand",
        "test_data": {
          "create": {},
          "update": {},
          "editor": {}
        }
      },
      {
      ...


## 📦 步骤7：下载模型（可选）

In [ ]:
# 打包模型用于下载
!cd /content/drive/MyDrive/gis-models && zip -r codellama-gis-lora.zip codellama-gis-lora/
print("✅ Model zipped!")
print(f"📦 Location: /content/drive/MyDrive/gis-models/codellama-gis-lora.zip")
print("💡 You can download it from Google Drive")

## 🎯 完成！

🎉 **恭喜！你已经完成了GIS代码生成模型的完整训练和评估流程！**

### 📊 你已完成：
1. ✅ 准备训练数据
2. ✅ 使用LoRA微调模型
3. ✅ 保存训练好的模型
4. ✅ 在测试集上评估模型性能
5. ✅ 下载模型到Google Drive

### 💻 本地使用模型

将模型下载到本地后，可以这样加载：

```python
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# 加载Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "path/to/codellama-gis-lora",
    trust_remote_code=True
)

# 加载基础模型
base_model = AutoModelForCausalLM.from_pretrained(
    "codellama/CodeLlama-7b-Instruct-hf",
    torch_dtype=torch.float16,
    device_map="auto"
)

# 加载LoRA权重
model = PeftModel.from_pretrained(
    base_model, 
    "path/to/codellama-gis-lora"
)

# 推理
prompt = "Create a new cable object..."
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=256)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)
```

---

### 🔧 训练参数调优建议：
- **T4 GPU**: batch_size=4, gradient_accumulation=4, 训练时间约4-6小时
- **A100 GPU**: batch_size=8-12, gradient_accumulation=2-4, 训练时间约1-2小时
- **显存不足**: 减小batch_size或max_length
- **提高质量**: 增加num_epochs、调整learning_rate、扩充训练数据

### 📚 相关文档：
- 本地加载指南: `load_model_local.py`
- Colab崩溃修复: `docs/COLAB_DRIVE_CRASH_FIX.md`
- 模型评估方法: `examples/evaluate_workflows.py`